### Setup

In [1]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 25kB/s 
     |████████████████████████████████| 12.8MB 39.3MB/s 
     |████████████████████████████████| 7.6MB 4.3MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [2]:
cd /content/drive/MyDrive/fourier_neural_operator/

/content/drive/MyDrive/fourier_neural_operator


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision.transforms import Normalize
import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from utilities3 import *
import random
import time

torch.manual_seed(0)
np.random.seed(0)

### Model

In [4]:
#Complex multiplication
def compl_mul1d(a, b):
    # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
    op = partial(torch.einsum, "bix,iox->box")
    return torch.stack([
        op(a[..., 0], b[..., 0]) - op(a[..., 1], b[..., 1]),
        op(a[..., 1], b[..., 0]) + op(a[..., 0], b[..., 1])
    ], dim=-1)

################################################################
#  1d fourier layer
################################################################
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  #Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels*out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, 2))

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.rfft(x, 1, normalized=True, onesided=True)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.in_channels, x.size(-1)//2 + 1, 2, device=x.device)
        out_ft[:, :, :self.modes1] = compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)

        #Return to physical space
        x = torch.irfft(out_ft, 1, normalized=True, onesided=True, signal_sizes=(x.size(-1), ))
        return x

class SimpleBlock1d(nn.Module):
    def __init__(self, modes, width):
        super(SimpleBlock1d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """

        self.modes1 = modes
        self.width = width
        self.fc0 = nn.Linear(2, self.width) # input channel is 2: (a(x), x)

        self.conv0 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):

        x = self.fc0(x)
        x = x.permute(0, 2, 1)

        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

class Net1d(nn.Module):
    def __init__(self, modes, width):
        super(Net1d, self).__init__()

        """
        A wrapper function
        """

        self.conv1 = SimpleBlock1d(modes, width)

    def forward(self, x):
        x = self.conv1(x)
        return x.squeeze()

    def count_params(self):
        c = 0
        for p in self.parameters():
            c += reduce(operator.mul, list(p.size()))

        return c

In [5]:
################################################################
#  Load Data
################################################################

dataloader = MatReader('data/burgers_data_R10.mat')

### Training

In [ ]:
################################################################
#  Training configurations
################################################################

ntrain = 1000
ntest = 100

sub = 2**3 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate
s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

modes = 16
width = 64

In [ ]:
################################################################
# Process training and testing dataset
################################################################

x_data = dataloader.read_field('a')[:,::sub]
y_data = dataloader.read_field('u')[:,::sub]

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]

# cat the locations information
grid = np.linspace(0, 2*np.pi, s).reshape(1, s, 1)
grid = torch.tensor(grid, dtype=torch.float)
x_train = torch.cat([x_train.reshape(ntrain,s,1), grid.repeat(ntrain,1,1)], dim=2)
x_test = torch.cat([x_test.reshape(ntest,s,1), grid.repeat(ntest,1,1)], dim=2)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

# model
model = Net1d(modes, width).cuda()
print(model.count_params())

549569


In [ ]:
################################################################
# training and evaluation
################################################################
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x)

        mse = F.mse_loss(out, y, reduction='mean')
        # mse.backward()
        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_mse += mse.item()
        train_l2 += l2.item()

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x)
            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_mse /= len(train_loader)
    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_mse, train_l2, test_l2)

if not os.path.exists(os.path.join(os.getcwd(), 'model')):
    os.makedirs(os.path.join(os.getcwd(), 'model'))
# torch.save(model, 'model/ns_fourier_burgers_8192')
pred = torch.zeros(y_test.shape)
index = 0
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=1, shuffle=False)
with torch.no_grad():
    for x, y in test_loader:
        test_l2 = 0
        x, y = x.cuda(), y.cuda()

        out = model(x)
        pred[index] = out

        test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
        print(index, test_l2)
        index = index + 1

# scipy.io.savemat('pred/burger_test.mat', mdict={'pred': pred.cpu().numpy()})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: The function torch.irfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.ifft or torch.fft.irfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:602.)


0 0.683003900000017 0.053555431168060746 0.36142572700977327 0.07761620402336121
1 0.6584615460000123 0.0014719982183305546 0.07033879584074021 0.058627632856369016
2 0.6568116039999836 0.0008255719099543057 0.0588114030957222 0.049599030017852784
3 0.6451836179999759 0.0007155934677575714 0.05558074927330017 0.03289741337299347
4 0.6549986200000149 0.0004070465952099767 0.04039650100469589 0.02958395779132843
5 0.650348092999991 0.0002657903070212342 0.03262121111154556 0.04327208757400513
6 0.6517683949999764 0.00030869538080878554 0.03585965767502785 0.03563477098941803
7 0.6436688210000057 0.00024819864840537777 0.03188886937499046 0.028739259839057923
8 0.6493157990000213 0.0002054120018146932 0.02987063756585121 0.016396253407001495
9 0.6486867790000019 0.00027883256880159026 0.03329298895597458 0.0273971688747406
10 0.6698317330000236 0.00022578369971597566 0.030902105927467345 0.024421341121196746
11 0.6433443020000027 0.00027449911955045535 0.03564715880155563 0.02746433198451

NameError: ignored

### Inverse

In [6]:
################################################################
#  inverse configurations
################################################################
ntrain = 1000

sub = 2**3 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate
s = h

learning_rate = 0.1
coef = 0.0001

epochs = 1500
step_size = 100
gamma = 0.5

In [7]:
################################################################
# Process inverse dataset
################################################################

x_data = dataloader.read_field('a')[:,::sub]
y_data = dataloader.read_field('u')[:,::sub]

index = random.randint(0,ntrain)
x_groundtruth = x_data[index,:]
y = y_data[index,:]

grid = np.linspace(0, 2*np.pi, s).reshape(1, s, 1)
grid = torch.tensor(grid, dtype=torch.float)
x_groundtruth = torch.cat([x_groundtruth.reshape(1,s,1), grid.repeat(1,1,1)], dim=2)
y = y.reshape(1,1024)

In [8]:
################################################################
# load model
################################################################

PATH = 'model/ns_fourier_burgers_8192'
model = torch.load(PATH)

In [9]:
################################################################
# Inverse
################################################################

#randomly generate parameters
x = torch.rand(x_groundtruth.shape, requires_grad=True, device="cuda")
# x = torch.norm(F.normalize(x_groundtruth, p=2, dim=1)).weight * torch.rand(x_groundtruth.shape, requires_grad=True).weight
# print(x)
# exit()
y, x_groundtruth = y.cuda(), x_groundtruth.cuda()
lr = 0.1
optimizer = torch.optim.SGD([x], lr=lr, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
myloss = nn.MSELoss()

t0 = time.time()

# x_first = x.clone()

for i in range(epochs):

    optimizer.zero_grad()
    output = model(x)
    loss = myloss(output.view(1, -1), y.view(1, -1))
    loss_groundtruth = myloss(x.view(1, -1), x_groundtruth.view(1, -1))

    loss.backward(retain_graph=True)
    loss_groundtruth.backward()
    
    optimizer.step()
    scheduler.step()

    print(i, loss.item(), loss_groundtruth.item())
    model.eval()


    # noise generation
    # noise = x.clone()
    # noise = coef * learning_rate * torch.norm(noise) * torch.randn(x.shape, requires_grad=False, device="cuda")
    # x = x + noise

# x_final = x.clone()

t1 = time.time()

print((t1-t0)/epochs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: The function torch.irfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.ifft or torch.fft.irfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:602.)


0 0.05378083884716034 5.422390937805176
1 0.05376109480857849 5.421321868896484
2 0.05372362956404686 5.419291973114014
3 0.0536702424287796 5.416397571563721
4 0.05360257253050804 5.412725448608398
5 0.053522028028964996 5.408355712890625
6 0.05342984199523926 5.4033589363098145
7 0.053327228873968124 5.397799491882324
8 0.053214795887470245 5.3917341232299805
9 0.05309375002980232 5.38521671295166
10 0.05296513810753822 5.378293991088867
11 0.052829522639513016 5.371006965637207
12 0.052687741816043854 5.363395690917969
13 0.05254032462835312 5.355493545532227
14 0.05238768085837364 5.347332000732422
15 0.052230626344680786 5.338937759399414
16 0.052069589495658875 5.3303375244140625
17 0.05190514773130417 5.3215532302856445
18 0.05173758044838905 5.312604904174805
19 0.05156749114394188 5.303512096405029
20 0.05139493942260742 5.294290542602539
21 0.0512201264500618 5.284955024719238
22 0.05104263871908188 5.275519371032715
23 0.05086318030953407 5.265995502471924
24 0.0506820306181